This notebook calculates the distance to the nearest 10^5.5 and 10^ 6.5 Msun molecular cloud from the locations of each of the SNe located within 150pc maps.

In [1]:
#import packages and initialize settings
import time
import os
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as pyfits
from astropy.io import ascii
from astropy.table import Table
from progressbar import progressbar
from astropy.wcs import WCS
from reproject import reproject_interp

import sys
sys.path.append('/home/mayker.1/Desktop/PythonFunctions')
from FindNearestMC import nearestMCMethod

In [2]:
# Unpack master table to get SNe information
database = Table.read('../Data/3.SNe+GalData.csv', format = "csv")

centerCoords = [(database["orient_ra"][i], database["orient_dec"][i]) for i in range(len(database))]    
SNe, SNra, SNdec  = database["SN_name"], database["SN_ra"], database["SN_dec"]
database[0:1]

galaxy,SN_name,SN_type,SN_ra,SN_dec,SN_xCoord,SN_yCoord,SN_co21int,SN_co21err,SN_co21ew,map_beamsize,orient_ra,orient_dec,dist,orient_posang,orient_incl,size_r25,telOrient,ImageFile150,ErrFile150,AlphaCOFile
str8,str11,str12,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str9,str112,str113,str74
circinus,SN1996cr,II,213.2918,-65.3457,86.56761411031889,84.63433198287285,186.84804,2.137819,34.56537,0.002032163908078,213.29124,-65.33915,4.2,36.74,64.3,262.49515,7m+tp,/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/circinus/circinus_7m+tp_co21_150pc_broad_mom0.fits,/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/circinus/circinus_7m+tp_co21_150pc_broad_emom0.fits,/data/kant/0/sun.1608/PHANGS/ALMA/alphaCO/ESO097-013_alphaCO21_Sun+20.fits


In [3]:
SNra  = np.array(SNra)
SNdec = np.array(SNdec)
massCutA = 10**5.5
massCutB = 10**6.5


[None, array([0.01364605]), array([0.55655997]), array([0.01321117]), array([0.00491407]), array([0.01275901]), array([0.1489542]), array([0.19534568]), array([0.35979567]), array([0.01287783]), array([0.01284268]), array([1.22508167]), array([0.01592836]), array([0.00761477]), array([0.00716833]), array([0.00421925]), None, array([0.07117317]), array([0.00321098]), array([0.0076644]), array([0.58246467]), array([0.01154423]), array([0.00611075]), array([0.00733605]), array([0.01245564]), array([0.01044295]), array([0.0040826]), array([0.00808865]), array([0.01097162]), array([0.02066585]), array([0.08586097]), array([0.03508382]), array([0.02765949]), array([0.0155404]), array([0.03955008]), array([0.02026862]), array([0.2480944]), array([0.10081377]), array([0.0106565]), array([0.00660132]), array([0.06650442]), array([0.01621761]), array([0.03209613]), array([0.38714994]), array([0.00946752]), array([0.29470551]), array([0.10821323]), array([0.37426845]), None, None, array([5800.10100662]), array([0.01400941]), array([0.01271513]), array([0.00453499]), array([0.00497956]), array([0.00238751]), array([0.05432539]), array([0.00023186]), array([0.0098961]), array([0.01070017]), array([0.99123115])]

[0.03220571, 0.03057009, 0.31916941, 0.75480179, 0.02805144]

In [4]:
gal_group = database.group_by("galaxy")
# print(gal_group)

In [5]:
gal_group.groups.indices

array([ 0,  1,  2,  3,  5,  6,  7,  9, 12, 13, 14, 18, 20, 21, 22, 23, 25,
       26, 31, 35, 42, 48, 49, 50, 52, 53, 55, 56, 58, 63, 64, 65, 66])

In [6]:
# remove duplicate galaxies, grouping SNe together in their own lists.

gal_group = database.group_by("galaxy")

SNr, SNd, SNa = [],[],[]
gala, imgs, errs, alco, cocs, pans, incl, gdis, SNrs, SNdc, SNea = [],[],[],[],[],[],[],[],[],[],[]

for i in range(len(database)):
    gal = database["galaxy"][i]
    img = database["ImageFile150"][i]
    ers = database["ErrFile150"][i]
    aco = database["AlphaCOFile"][i]
    ccs = centerCoords[i]
    pas = database["orient_posang"][i]
    inc = database["orient_incl"][i]
    gds = database["dist"][i]
    SNr.append(SNra[i])
    SNd.append(SNdec[i])
    SNa.append(SNe[i])
    
    if (i+1) in gal_group.groups.indices:
        gala.append(gal)
        imgs.append(img)
        errs.append(ers)
        alco.append(aco)
        cocs.append(ccs)
        pans.append(pas)
        incl.append(inc)
        gdis.append(gds)
        SNrs.append(SNr)
        SNdc.append(SNd)
        SNea.append(SNa) 
        SNr, SNd, SNa = [],[],[]
    else: pass

In [7]:
#for i in range(len(gal_group.groups.indices)-1):
print(SNea)

[['SN1996cr'], ['SN1940E'], ['SN2019qyl'], ['SN2019krl', 'SN2013ej'], ['SN2018ivc'], ['SN1995V'], ['SN1999eu', 'SN1992bd'], ['SN2001du', 'SN1983V', 'SN2012fr'], ['SN1985P'], ['SN1935C'], ['SN1984J', 'SN1986L', 'SN2005df', 'SN2009ib'], ['SN2010el', 'ASASSN-14ha'], ['SN1999em'], ['SN2017gax'], ['SN1993Z'], ['SN2003jg', 'SN2008eh'], ['SN2012A'], ['SN2009hd', 'SN1997bs', 'SN1973R', 'SN1989B', 'SN2016cok'], ['SN1972Q', 'SN1967H', 'SN2014L', 'SN1986I'], ['SN2006ov', 'SN2020jfo', 'SN2014dt', 'SN1964F', 'SN1961I', 'SN1999gn', 'SN1926A'], ['SN2006X', 'SN1959E', 'SN2020oi', 'SN1979C', 'SN1901B', 'SN2019ehk'], ['SN2012cg'], ['SN2020nvb'], ['SN1988M', 'SN1960F'], ['SN1981B'], ['SN1988A', 'SN1989M'], ['SN2011ja'], ['SN1986G', 'SN2016adj'], ['SN1957D', 'SN1923A', 'SN1950B', 'SN1983N', 'SN1968L'], ['SN2007it'], ['SN2005at'], ['SN2008bk']]


In [15]:

nearest55, nearest65 = [],[]
for i in range(len(gala)):
    if incl[i] == 90.0:
        incl[i]=80.0
    near55, near65 = nearestMCMethod(gala[i], imgs[i], errs[i], alco[i], cocs[i], pans[i], incl[i], gdis[i], SNrs[i], SNdc[i], SNea[i])
    nearest55.append(near55)
    nearest65.append(near65)

at Mass cutoff A for circinus
Nearest 55 [32.20571075] circinus
Nearest 65 [32.20571075] circinus
done with circinus   ['SN1996cr']
at Mass cutoff A for ngc0253
Nearest 55 [30.57009157] ngc0253
Nearest 65 [30.57009157] ngc0253
done with ngc0253   ['SN1940E']
at Mass cutoff A for ngc0300
Nearest 55 [291.61613386] ngc0300
Nearest 65 nan ngc0300
done with ngc0300   ['SN2019qyl']
at Mass cutoff A for ngc0628
Nearest 55 [754.80178883  97.87335683] ngc0628
Nearest 65 [1403.88983182 1914.40658196] ngc0628
done with ngc0628   ['SN2019krl', 'SN2013ej']
No file for  ngc1068
at Mass cutoff A for ngc1087
Nearest 55 [13.6460475] ngc1087
Nearest 65 [662.29661955] ngc1087
done with ngc1087   ['SN1995V']
at Mass cutoff A for ngc1097
Nearest 55 [279.64329546  13.21116559] ngc1097
Nearest 65 [6549.95976975   13.21116559] ngc1097
done with ngc1097   ['SN1999eu', 'SN1992bd']
at Mass cutoff A for ngc1365
Nearest 55 [175.45337986  12.75901358 362.78570368] ngc1365
Nearest 65 [2918.12897775 1149.63976938 335

In [20]:
n55A = []
for i in range(len(nearest55)):
    if(type(nearest55[i])!= np.float64 and type(nearest55[i]) != float):
        for j in range(len(nearest55[i])):
            n55A.append(nearest55[i][j])
    else:
        n55A.append(nearest55[i])
#print(n55A)

n65A = []
for i in range(len(nearest65)):
    if(type(nearest65[i])!= np.float64 and type(nearest65[i]) != float):
        for j in range(len(nearest65[i])):
            n65A.append(nearest65[i][j])
    else:
        n65A.append(nearest65[i])
#print(n65A)

In [17]:
with open("../Data/NearestMCtoSNe.txt", "w") as textFile:
    print("# Nearest55MassArrayDistances, Nearest65MassArrayDistances (in pc)", file=textFile)
    print(n55A, n65A, file=textFile)